In [ ]:
# imports

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from typing import Union, Any
import json
import ast
import os

In [7]:
# check working directory

current_directory = os.getcwd()
print("Current Working Directory:", current_directory)



Current Working Directory: /Users/gavinbarro/Documents/VT_programming/Fall_2025/CS-5805-Machine-Learning/CinemaSuccessSage/notebooks


In [ ]:
# load merged data
merged_df = pd.read_parquet("../data/processed/merged_movies.parquet")

# handle missing values
merged_df['keywords'] = merged_df['keywords'].fillna('[]')
merged_df['belongs_to_collection'] = merged_df['belongs_to_collection'].fillna(0)  # 0 for non franchise
merged_df['homepage_tmdb'] = merged_df['homepage_tmdb'].fillna('')  # empty string for non critical
merged_df['tagline_tmdb'] = merged_df['tagline_tmdb'].fillna('')
merged_df = merged_df.dropna(subset=['release_date_tmdb', 'genres_tmdb'])  # drop rare core misses

In [19]:
# parse JSON like columns (genres, cast, crew as strings should yield lists/dicts)

def safe_parse(x: str) -> Any:
    try:
        return ast.literal_eval(x) if isinstance(x, str) else []
    except:
        return []

# apply safe parse to columns
merged_df['genres_tmdb'] = merged_df['genres_tmdb'].apply(safe_parse)
merged_df['cast'] = merged_df['cast'].apply(safe_parse)
merged_df['crew'] = merged_df['crew'].apply(safe_parse)
merged_df['keywords'] = merged_df['keywords'].apply(safe_parse)
merged_df['production_companies_tmdb'] = merged_df['production_companies_tmdb'].apply(safe_parse)
merged_df['production_countries_tmdb'] = merged_df['production_countries_tmdb'].apply(safe_parse)
merged_df['spoken_languages_tmdb'] = merged_df['spoken_languages_tmdb'].apply(safe_parse)

# basic extraction examples - expand later
merged_df['num_genres'] = merged_df['genres_tmdb'].apply(len)
merged_df['primary_genre'] = merged_df['genres_tmdb'].apply(lambda x: x[0]['name'] if x else 'Unknown')
merged_df['num_cast'] = merged_df['cast'].apply(len)
merged_df['num_crew'] = merged_df['crew'].apply(len)


In [20]:
# Date Handling and Derived Features


# basic extraction examples -- number of genres, top genre, cast size
merged_df['num_genres'] = merged_df['genres_tmdb'].apply(len)
merged_df['primary_genre'] = merged_df['genres_tmdb'].apply(lambda x: x[0]['name'] if x else 'Unknown')
merged_df['num_cast'] = merged_df['cast'].apply(len)
merged_df['num_crew'] = merged_df['crew'].apply(len)

# parse release_date to datetime, extract year/month/season
merged_df['release_date_tmdb'] = pd.to_datetime(merged_df['release_date_tmdb'], errors='coerce')
merged_df['release_year'] = merged_df['release_date_tmdb'].dt.year
merged_df['release_month'] = merged_df['release_date_tmdb'].dt.month
merged_df['release_season'] = merged_df['release_month'].apply(lambda m: 'Winter' if m in [12,1,2] else 'Spring' if m in [3,4,5] else 'Summer' if m in [6,7,8] else 'Fall')

In [21]:
# handle skews/outliers in targets/numerics (from EDA: revenue skewed)

# log-transform revenue (add 1 to avoid log(0))
merged_df['log_revenue'] = np.log1p(merged_df['revenue_tmdb'])

# ror ratings, no transform needed (normal dist) while clip extremes
merged_df['avg_user_rating'] = merged_df['avg_user_rating'].clip(1, 5)

In [22]:
# define features and rargets

# numerical: scale
num_features = ['budget_tmdb', 'runtime_tmdb', 'popularity_tmdb', 'vote_count_tmdb', 'user_rating_count', 'num_genres', 'num_cast', 'num_crew', 'release_year']

# categorical: encode (onehot for multilabel like genres, label for single like language)
cat_features = ['primary_genre', 'original_language_tmdb', 'release_season', 'adult_tmdb']

# targets
target_revenue = 'log_revenue' 
target_rating = 'avg_user_rating'

In [23]:
# preprocessing pipeline
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), num_features), 
        ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), cat_features)
    ])

# train/test split
X = merged_df[num_features + cat_features]
y_revenue = merged_df[target_revenue]
y_rating = merged_df[target_rating]

X_train, X_test, y_revenue_train, y_revenue_test = train_test_split(X, y_revenue, test_size=0.2, random_state=42)
_, _, y_rating_train, y_rating_test = train_test_split(X, y_rating, test_size=0.2, random_state=42)


In [24]:
# apply preprocessor
X_train_processed = preprocessor.fit_transform(X_train)
X_test_processed = preprocessor.transform(X_test)

# convert back to DF for ease (with feature names)
feature_names = preprocessor.get_feature_names_out()
X_train_df = pd.DataFrame(X_train_processed, columns=feature_names, index=X_train.index)
X_test_df = pd.DataFrame(X_test_processed, columns=feature_names, index=X_test.index)

# combine with targets for saving
train_df = X_train_df.assign(revenue=y_revenue_train, rating=y_rating_train)
test_df = X_test_df.assign(revenue=y_revenue_test, rating=y_rating_test)

# save processed splits
train_df.to_parquet('../data/processed/processed_train.parquet', index=False)
test_df.to_parquet('../data/processed/processed_test.parquet', index=False)

In [25]:
# inspect results

print(f"Train Shape: {train_df.shape}, Test Shape: {test_df.shape}")
print(train_df.head())
print("\nProcessed Data Types:\n", train_df.dtypes)
print("\nMissing Values in Train (%):\n", (train_df.isnull().mean() * 100).round(2))

Train Shape: (4760, 49), Test Shape: (1190, 49)
      num__budget_tmdb  num__runtime_tmdb  num__popularity_tmdb  \
1117          0.403193           1.868612              0.275292   
4327         -0.674719           0.350889             -0.169126   
3181          0.566927           0.008177              0.080218   
642          -0.279030           0.252971              1.095789   
5898         -0.787968          -1.411629             -0.781579   

      num__vote_count_tmdb  num__user_rating_count  num__num_genres  \
1117              0.336878                0.905592              0.0   
4327             -0.520796               -0.448970              0.0   
3181             -0.390701                1.664422              0.0   
642               0.939424                1.254829              0.0   
5898             -0.615465               -0.476542              0.0   

      num__num_cast  num__num_crew  num__release_year  \
1117            0.0            0.0          -0.072528   
4327    